<p align="center"><strong><span style="font-size: 50px;">PROYECTO INDIVIDUAL NÚMERO UNO MACHINE LEARNING OPERATIONS (MLOps)</span></strong></p>


LIBRERIAS NECESARIAS PARA TRABAJAR

In [1]:
import pandas as pd
import json
import re
import ast
import numpy as np

## ARCHIVO REVIEWS Y TRATAMIENTO DE DATOS

Apertura del archivo 'australian_user_reviews.json' a dataframe tipo pandas

In [2]:
# Leer el archivo JSON línea por línea y cargarlo en una lista
data = []
with open('australian_user_reviews.json', 'r', encoding='utf-8') as file:
    for line in file:
        try:
            data.append(eval(line))
        except Exception as e:
            print(f"Error al procesar una línea: {str(e)}")

# Crear un DataFrame a partir de la lista de datos
dfReviews = pd.DataFrame(data)

# Mostrar las primeras filas del DataFrame
dfReviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Eliminación de duplicados

In [3]:
dfReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [4]:
dfReviews['user_id'].value_counts()

user_id
76561198027488037    3
76561198045953692    3
76561198051777058    3
76561198100326818    3
blablabla174         3
                    ..
SakurasouNo          1
goneckahorse         1
coutlindo            1
superdedicated       1
LydiaMorley          1
Name: count, Length: 25485, dtype: int64

In [5]:
filtro = dfReviews.loc[dfReviews['user_id'] == '76561198027488037']

Encontramos que al hacer el filtro tenemos duplicados en id_user que son similares en todo, procedemos a eliminar

In [6]:
filtro

,user_id,user_url,reviews
6935,76561198027488037,http://steamcommunity.com/profiles/76561198027...,"[{'funny': '', 'posted': 'Posted May 12.', 'la..."
6936,76561198027488037,http://steamcommunity.com/profiles/76561198027...,"[{'funny': '', 'posted': 'Posted May 12.', 'la..."
15693,76561198027488037,http://steamcommunity.com/profiles/76561198027...,"[{'funny': '', 'posted': 'Posted May 12.', 'la..."


In [7]:
dfReviews.drop_duplicates(subset='user_id', inplace=True)

In [8]:
dfReviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25485 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25485 non-null  object
 1   user_url  25485 non-null  object
 2   reviews   25485 non-null  object
dtypes: object(3)
memory usage: 796.4+ KB


Reset del index para secuencia de organizar

In [9]:
dfReviews.reset_index(drop= True, inplace= True)

In [10]:
dfReviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [11]:
dfReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25485 entries, 0 to 25484
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25485 non-null  object
 1   user_url  25485 non-null  object
 2   reviews   25485 non-null  object
dtypes: object(3)
memory usage: 597.4+ KB


Generamos una copia para proteger el dataframe original

In [12]:
dfDReviews = dfReviews['reviews'].copy()

Como tenemos diccionarioarios en la columna review los abrimos por medio del modulo explode

In [13]:
# Normalizar la columna 'reviews' 
dfRevDesan = dfReviews.explode('reviews')
# reseteo el index
dfRevDesan.reset_index(drop=True, inplace=True)

Se desanida y se va generando las nuevas columnas para que almacene la información correspondiente en un dataframe individual por medio de un ciclo for

In [14]:
# Lista para almacenar los DataFrames individuales de cada fila
dfs = []

# Itera sobre la columna 'items' y desanida cada diccionario en un DataFrame individual
for index, row in dfReviews.iterrows():
    items = row['reviews']
    if items:
        df_item = pd.DataFrame(items)
        dfs.append(df_item)

# Concatena todos los DataFrames individuales en uno solo
dfReviewsDesanidado = pd.concat(dfs, ignore_index=True)

Creación del nuevo dataframe 

In [15]:
# Combina los DataFrames dfItemsDesan y df_desanidado en uno nuevo llamado dfItemsDesanidado
dfComReviews = pd.concat([dfRevDesan, dfReviewsDesanidado], axis=1)

Estructura del nuevo dataframe 

In [16]:
dfComReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58458 entries, 0 to 58457
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      58458 non-null  object
 1   user_url     58458 non-null  object
 2   reviews      58430 non-null  object
 3   funny        58430 non-null  object
 4   posted       58430 non-null  object
 5   last_edited  58430 non-null  object
 6   item_id      58430 non-null  object
 7   helpful      58430 non-null  object
 8   recommend    58430 non-null  object
 9   review       58430 non-null  object
dtypes: object(10)
memory usage: 4.5+ MB


In [17]:
dfComReviews.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Eliminacion de la columna review que fue la desanidada adicional se borran d ela nueva columna item_id las filas repetidas y que a su vez repercuta en las demás columnas 

In [18]:
dfComReviews=dfComReviews.drop('reviews', axis=1)

In [19]:
dfComReviews.dropna(subset=["item_id"], inplace=True)

Llenamos la columna funny los NaN con cero(0) y tranformamos los datos a tipo entero

In [20]:
# Utilizar una expresión regular para extraer los números y mantener los ceros
dfComReviews['funny'] = dfComReviews['funny'].str.extract(r'(\d+)').fillna(0).astype(int)

Tenemos la columna posted vamos a extraer una nueva columna que referencie el año nada más adicional se eliminara el texto para quedarnos solo con la fecha

In [21]:
import pandas as pd
import re

# Función para extraer el año de la columna 'posted'
def extract_year(posted):
    match = re.search(r'\d{4}', posted)  # Busca un patrón de cuatro dígitos que representa el año
    if match:
        return int(match.group())  # Devuelve el año como entero
    else:
        return None  # Si no se encuentra un año, devuelve NaN

# Aplica la función extract_year a la columna 'posted' y crea la nueva columna 'postedAnios'
dfComReviews['postedAnios'] = dfComReviews['posted'].apply(extract_year)

# Muestra las primeras filas del DataFrame resultante
#print(resultadoFinal.head())


Se eliminan los NaN de la nueva columna y que repercuta en las demás columnas

In [22]:
#Eliminar filas con NaN en la columna 'postedAnios' y las filas correspondientes en otras columnas
dfComReviews = dfComReviews.dropna(subset=['postedAnios'])

In [23]:
dfComReviews.reset_index(drop= True, inplace= True)

Eliminamos columnas que para nuestro críterio no son utiles

In [24]:
columns = ['last_edited','helpful','posted']
dfComReviews = dfComReviews.drop(columns,axis=1)

Cambiamos la estructura de la columna de la columna recommend a uno(1) y cero(0)

In [25]:
# Reemplazar 'True' por 1 y 'False' por 0 en la columna 'recommend'
dfComReviews['recommend'] = dfComReviews['recommend'].replace({True: 1, False: 0})

Cambiamos algunas columnas a tipo de dato entero

In [26]:
columns = ['funny','recommend','postedAnios','item_id']
dfComReviews[columns] = dfComReviews[columns].astype(int)

In [27]:
dfComReviews.head()

,user_id,user_url,funny,item_id,recommend,review,postedAnios
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,1250,1,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,22200,1,It's unique and worth a playthrough.,2011
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,43110,1,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,http://steamcommunity.com/id/js41637,0,251610,1,I know what you think when you see this title ...,2014
4,js41637,http://steamcommunity.com/id/js41637,0,227300,1,For a simple (it's actually not all that simpl...,2013


In [28]:
dfComReviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48498 entries, 0 to 48497
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      48498 non-null  object
 1   user_url     48498 non-null  object
 2   funny        48498 non-null  int32 
 3   item_id      48498 non-null  int32 
 4   recommend    48498 non-null  int32 
 5   review       48498 non-null  object
 6   postedAnios  48498 non-null  int32 
dtypes: int32(4), object(3)
memory usage: 1.9+ MB


### Generamos un nuevo archivo csv con los datos revisados y reparados

In [29]:
dfComReviews.to_csv('dfReviewsListo', index=False) 

## ARCHIVO ITEMS Y TRATAMIENTO DE DATOS

Apertura del archivo 'australian_users_items.json'

In [30]:
items = []
with open('australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        items.append(ast.literal_eval(line))

# Crear el DataFrame anidado
dfItems = pd.DataFrame(items) 


In [31]:
dfItems.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [32]:
dfItems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


Filtro de usuarios repetidos

In [33]:
filter = dfItems['user_id'].value_counts()

In [34]:
filter

user_id
X03-Suits            3
76561198027488037    3
76561198100326818    3
76561198309337430    3
76561198051777058    3
                    ..
8392158              1
76561198056804863    1
SparklezTheTurtle    1
76561198019707497    1
edward_tremethick    1
Name: count, Length: 87626, dtype: int64

In [35]:
dfItems[dfItems['user_id']=='X03-Suits']

,user_id,items_count,steam_id,user_url,items
14832,X03-Suits,141,76561198085989695,http://steamcommunity.com/id/X03-Suits,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
14837,X03-Suits,141,76561198085989695,http://steamcommunity.com/id/X03-Suits,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
36239,X03-Suits,141,76561198085989695,http://steamcommunity.com/id/X03-Suits,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


Eliminación de duplicados

In [36]:
dfItems.drop_duplicates(subset='user_id', inplace=True)

In [37]:
dfItems.reset_index(drop=True, inplace= True)

In [38]:
dfItems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87626 entries, 0 to 87625
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      87626 non-null  object
 1   items_count  87626 non-null  int64 
 2   steam_id     87626 non-null  object
 3   user_url     87626 non-null  object
 4   items        87626 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.3+ MB


Generamos uan copia para proteger los datos del dataframe principal


In [39]:
dfDItems = dfItems['items'].copy()

In [40]:
dfItems.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [41]:
dfDItems

0        [{'item_id': '10', 'item_name': 'Counter-Strik...
1        [{'item_id': '10', 'item_name': 'Counter-Strik...
2        [{'item_id': '1200', 'item_name': 'Red Orchest...
3        [{'item_id': '10', 'item_name': 'Counter-Strik...
4        [{'item_id': '300', 'item_name': 'Day of Defea...
                               ...                        
87621    [{'item_id': '413850', 'item_name': 'CS:GO Pla...
87622    [{'item_id': '11020', 'item_name': 'TrackMania...
87623                                                   []
87624    [{'item_id': '304930', 'item_name': 'Unturned'...
87625                                                   []
Name: items, Length: 87626, dtype: object

Vamos a realizar los mismos procesos que se generaron para desanidar el archivo de reviews pero ahora en Items

In [42]:
# Normalizar la columna 'items' 
dfItemDesan = dfItems.explode('items')
# reseteo el index
dfItemDesan.reset_index(drop=True, inplace=True)

# eliminamos los vacios de la columna review
#dfItemsDesan = dfItems.dropna(subset=['items'])

In [43]:
dfItemDesan.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [44]:
# Lista para almacenar los DataFrames individuales de cada fila
dfs = []

# Itera sobre la columna 'items' y desanida cada diccionario en un DataFrame individual
for index, row in dfItems.iterrows():
    items = row['items']
    if items:
        df_item = pd.DataFrame(items)
        dfs.append(df_item)

# Concatena todos los DataFrames individuales en uno solo
dfItemsDesanidado = pd.concat(dfs, ignore_index=True)

In [45]:
dfItemsDesanidado.head()

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0


In [46]:
dfItemsDesanidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5094082 entries, 0 to 5094081
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
dtypes: int64(2), object(2)
memory usage: 155.5+ MB


In [47]:
# Combina los DataFrames dfItemsDesan y df_desanidado en uno nuevo llamado dfItemsDesanidado
dfComItems = pd.concat([dfItemDesan, dfItemsDesanidado], axis=1)

In [48]:
dfComItems.head()

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50,Half-Life: Opposing Force,0.0,0.0


In [49]:
dfComItems=dfComItems.drop('items', axis=1)

In [50]:
dfComItems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110796 entries, 0 to 5110795
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 311.9+ MB


Se suma las columnas playtime_forever con playtime_2weeks y dejamos la playtimeTotal para garantizar el tiempo total

In [51]:
# Crear la nueva columna "playtimeTotal" que contiene la suma de "playtime_forever" y "playtime_2weeks"
dfComItems['playtimeTotal'] = dfComItems['playtime_forever'] + dfComItems['playtime_2weeks']

Busquedas de nulos y limpieza de las columnas y se selecciona las más adecuadas

In [52]:
# Calcula la cantidad de datos nulos por columna
nulos_por_columna = dfComItems.isnull().sum()

# Muestra la cantidad de datos nulos por columna
print(nulos_por_columna)

user_id                 0
items_count             0
steam_id                0
user_url                0
item_id             16714
item_name           16714
playtime_forever    16714
playtime_2weeks     16714
playtimeTotal       16714
dtype: int64


Tomamos las columnas diferentes a cero(0)

In [53]:
dfComItems = dfComItems[dfComItems['playtimeTotal'] != 0]

In [54]:
dfComItems.dropna(subset=["user_id"], inplace=True)

In [55]:
dfComItems = dfComItems.reset_index(drop=True)

Llenamos con la palabra 'Desconocido' la columna item_name los NaN

In [56]:
dfComItems["item_name"].fillna("Desconocido", inplace=True)

Ekiminación de los NaN en la columna item_id y a su vez se eliminan esos index en las demás columnas

In [57]:
dfComItems.dropna(subset=["item_id"], inplace=True)

Eliminación de columnas que no tendremos en cuenta según nuestro críterio

In [58]:
columnas_a_eliminar = ['playtime_forever', 'playtime_2weeks','steam_id','items_count']
dfComItems = dfComItems.drop(columnas_a_eliminar, axis=1)

Cambio de tipo de dato a entero en algunas columnas

In [59]:
columns = ['item_id','playtimeTotal']
dfComItems[columns] = dfComItems[columns].astype(int)

In [60]:
dfComItems.head()

,user_id,user_url,item_id,item_name,playtimeTotal
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,300,Day of Defeat: Source,4733
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,240,Counter-Strike: Source,1853
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3830,Psychonauts,333


In [61]:
dfComItems.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3246355 entries, 0 to 3246354
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        object
 1   user_url       object
 2   item_id        int32 
 3   item_name      object
 4   playtimeTotal  int32 
dtypes: int32(2), object(3)
memory usage: 123.8+ MB


### Generación de un archivo tipo csv con los datos de Items tratados y reparados 

In [62]:
dfComItems.to_csv('dfItemsListo', index=False) 

## ARCHIVO GAMES Y TRATAMIENTO DE DATOS

Apertura del archivo 'output_steam_games.json'

In [63]:
# Nombre del archivo JSON
file_name = 'output_steam_games.json'

# Lista para almacenar los registros del archivo JSON
data = []

# Abrir y cargar el archivo JSON
with open(file_name, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            record = json.loads(line)
            data.append(record)
        except json.JSONDecodeError:
            continue  # Omitir líneas inválidas en el archivo JSON

# Crear un DataFrame
dfGames = pd.DataFrame(data)

# Visualizar el DataFrame
dfGames.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
dfGames.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


In [65]:
dfGames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


Se hace una revisión de los datos a traves del indice por el total menos los non-null, se toma el dataframe a partir del index 88310 en adelante y se reinicia el index 

In [66]:
dfGames_subset = dfGames.iloc[88000:]

In [67]:
dfGames = dfGames.iloc[88310:]

In [68]:
dfGames.reset_index(drop=True, inplace=True)

Eliminación de los duplicados a través de la columna id, como tambien los id vacios. Comparación de dos columnas con información similar se comparan y se llenan en una sola columna llamada Titulo

In [69]:
dfGames = dfGames.drop_duplicates(subset=['id'])

In [70]:
dfGames = dfGames.dropna(subset=['id'])

In [71]:
dfGames['Titulo'] = dfGames['app_name'].combine_first(dfGames['title'])

filtro busqueda de NaN en la columna publisher

In [72]:
nan_count = dfGames['publisher'].isna().sum()
print("Cantidad de NaN en la columna 'publisher':", nan_count)

Cantidad de NaN en la columna 'publisher': 8051


Se llena en columnas donde esta vacio pero comparte información relevante con los demás con la palabra Desconocido

In [73]:
columns = ['publisher', 'Titulo', 'developer']
dfGames[columns] = dfGames[columns].fillna('Desconocido')

In [74]:
# Reemplazar listas vacías con ['Desconocido']
dfGames['genres'] = dfGames['genres'].apply(lambda x: x if isinstance(x, list) and len(x) > 0 else ['Desconocido'])

Cambio de formato a fecha 

In [75]:
dfGames['release_date'] = pd.to_datetime(dfGames['release_date'], errors='coerce')

In [76]:
# Reemplazar listas vacías con ['Desconocido']
dfGames['tags'] = dfGames['tags'].apply(lambda x: x if isinstance(x, list) and len(x) > 0 else ['Desconocido'])

Se busca las coincidencias para aplicar cambio a cero en la columna price, como también identificar otros tipos de datos que afectan y buscar la mejor estructura en su tratamiento 

In [77]:
dfGames.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Titulo
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,Lost Summoner Kitty
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Ironbound
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Real Pool 3D - Poolians
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,弹炸人2222
4,Desconocido,[Desconocido],Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaT,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,Desconocido,Log Challenge


In [78]:
price_types = dfGames['price'].unique()
print(price_types)   

[4.99 'Free To Play' 'Free to Play' 0.99 2.99 3.99 9.99 18.99 29.99 nan
 'Free' 10.99 1.59 14.99 1.99 59.99 8.99 6.99 7.99 39.99 19.99 7.49 12.99
 5.99 2.49 15.99 1.25 24.99 17.99 61.99 3.49 11.99 13.99 'Free Demo'
 'Play for Free!' 34.99 74.76 1.49 32.99 99.99 14.95 69.99 16.99 79.99
 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71 'Install Now'
 21.99 89.99 'Play WARMACHINE: Tactics Demo' 0.98 139.92 4.29 64.99
 'Free Mod' 54.99 74.99 'Install Theme' 0.89 'Third-party' 0.5 'Play Now'
 299.99 1.29 3.0 15.0 5.49 23.99 49.0 20.99 10.93 1.39
 'Free HITMAN™ Holiday Pack' 36.99 4.49 2.0 4.0 9.0 234.99 1.95 1.5 199.0
 189.0 6.66 27.99 10.49 129.99 179.0 26.99 399.99 31.99 399.0 20.0 40.0
 3.33 199.99 22.99 320.0 38.85 71.7 59.95 995.0 27.49 3.39 6.0 19.95
 499.99 16.06 4.68 131.4 44.98 202.76 1.0 2.3 0.95 172.24 249.99 2.97
 10.96 10.0 30.0 2.66 6.48 19.29 11.15 18.9 2.89 'Play the Demo' 99.0
 87.94 599.0 8.98 9.69 0.49 9.98 9.95 7.0 'Starting at $499.00'
 'Starting at $449.00' 12.8

In [79]:
# Filtra las cadenas de texto que no contienen números en la columna "price"
non_numeric_prices = dfGames['price'].dropna().apply(lambda x: x if not re.search(r'\d', str(x)) else None).dropna().unique()

print(non_numeric_prices)

['Free To Play' 'Free to Play' 'Free' 'Free Demo' 'Play for Free!'
 'Install Now' 'Play WARMACHINE: Tactics Demo' 'Free Mod' 'Install Theme'
 'Third-party' 'Play Now' 'Free HITMAN™ Holiday Pack' 'Play the Demo'
 'Free to Try' 'Free Movie' 'Free to Use']


Cambiamos estos datos a cero en la columna price pues su relación fue previamente revisada row por row

In [80]:
# Lista de etiquetas recomendadas para cambiar a 0
etiquetas_cero = [
    'Free To Play',
    'Free to Play',
    'Free',
    'Free Demo',
    'Play for Free!',
    'Play WARMACHINE: Tactics Demo',
    'Free Mod',
    'Play Now',
    'Free HITMAN™ Holiday Pack',
    'Play the Demo',
    'Free to Try',
    'Install Now',
    'Install Theme',
    'Third-party',
    'Free Movie',	
    'Free to Use'
]

# Reemplazar las etiquetas recomendadas por 0 en la columna 'price'
dfGames['price'] = dfGames['price'].apply(lambda x: 0 if x in etiquetas_cero else x)


Datos especiales en donde solo vamos a tomar su parte númerica

In [81]:
filtro_install_now = dfGames[dfGames['price'] == 'Starting at $499.00']
filtro_install_now

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Titulo
25000,Desconocido,[Desconocido],Syber Steam Machine,Syber Steam Machine,http://store.steampowered.com/app/353420/Syber...,2015-11-10,"[Steam Machine, Hardware]",http://steamcommunity.com/app/353420/reviews/?...,NaN,Starting at $499.00,False,353420,Desconocido,Syber Steam Machine


In [82]:
# Cambiar el valor en la fila con índice 25000 en la columna "price"
dfGames.at[25000, 'price'] = '499.00'
dfGames.at[25001, 'price'] = '449.00'

Hacemos cambio de la forma y estructura del dato a cero(0) y uno(1)

In [83]:
# Reemplazar 'True' por 1 y 'False' por 0 en la columna 'early_access'
dfGames['early_access'] = dfGames['early_access'].replace({True: 1, False: 0})

Se genera una nueva columna llamada anios, para extraer de la columna release_year el año

In [84]:
# Crear la columna "anios" que contiene el año de "release_date"
dfGames['anios'] = dfGames['release_date'].dt.year

Eliminación de los NaN en la columna anios y que a su vez se eliminen esos index en las demás columnas 

In [85]:
dfGames.dropna(subset=['anios'], inplace=True)

In [86]:
dfGames.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Titulo,anios
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0,761140,Kotoshiro,Lost Summoner Kitty,2018.0
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0,0,643980,Secret Level SRL,Ironbound,2018.0
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0,0,670290,Poolians.com,Real Pool 3D - Poolians,2017.0
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0,767400,彼岸领域,弹炸人2222,2017.0
5,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0,772540,Trickjump Games Ltd,Battle Royale Trainer,2018.0


In [87]:
dfGames.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29781 entries, 0 to 32133
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   publisher     29781 non-null  object        
 1   genres        29781 non-null  object        
 2   app_name      29780 non-null  object        
 3   title         29780 non-null  object        
 4   url           29781 non-null  object        
 5   release_date  29781 non-null  datetime64[ns]
 6   tags          29781 non-null  object        
 7   reviews_url   29781 non-null  object        
 8   specs         29112 non-null  object        
 9   price         28780 non-null  object        
 10  early_access  29781 non-null  int64         
 11  id            29781 non-null  object        
 12  developer     29781 non-null  object        
 13  Titulo        29781 non-null  object        
 14  anios         29781 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1

Reset del index, renombramos las columnas, eliminamos las columnas que no tendremos encuenta, cambiamos el tipo de variable de algunas columnas según el críterio a trabajar

In [88]:
dfGames.reset_index(drop=True, inplace=True)

In [89]:
# Renombrar la columna "id" a "item_id"
dfGames.rename(columns={'id': 'item_id'}, inplace=True)

In [90]:
columns = ['publisher', 'app_name','title','release_date','specs','tags']
dfGames = dfGames.drop(columns, axis=1)

In [91]:
columns = ['anios','item_id','early_access']
dfGames[columns] = dfGames[columns].astype(int)

In [92]:
columns = ['price']
dfGames[columns] = dfGames[columns].astype(float)

In [93]:
dfGames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29781 entries, 0 to 29780
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        29781 non-null  object 
 1   url           29781 non-null  object 
 2   reviews_url   29781 non-null  object 
 3   price         28780 non-null  float64
 4   early_access  29781 non-null  int32  
 5   item_id       29781 non-null  int32  
 6   developer     29781 non-null  object 
 7   Titulo        29781 non-null  object 
 8   anios         29781 non-null  int32  
dtypes: float64(1), int32(3), object(5)
memory usage: 1.7+ MB


In [94]:
dfGames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29781 entries, 0 to 29780
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        29781 non-null  object 
 1   url           29781 non-null  object 
 2   reviews_url   29781 non-null  object 
 3   price         28780 non-null  float64
 4   early_access  29781 non-null  int32  
 5   item_id       29781 non-null  int32  
 6   developer     29781 non-null  object 
 7   Titulo        29781 non-null  object 
 8   anios         29781 non-null  int32  
dtypes: float64(1), int32(3), object(5)
memory usage: 1.7+ MB


## Generamos un nuevo archivo csv, con los datos de Games tratados y reparados

In [95]:
dfGames.to_csv('dfGamesListo', index=False) 

In [96]:
dfGames.head()

,genres,url,reviews_url,price,early_access,item_id,developer,Titulo,anios
0,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,http://steamcommunity.com/app/761140/reviews/?...,4.99,0,761140,Kotoshiro,Lost Summoner Kitty,2018
1,"[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,http://steamcommunity.com/app/643980/reviews/?...,0.00,0,643980,Secret Level SRL,Ironbound,2018
2,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,http://steamcommunity.com/app/670290/reviews/?...,0.00,0,670290,Poolians.com,Real Pool 3D - Poolians,2017
3,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,http://steamcommunity.com/app/767400/reviews/?...,0.99,0,767400,彼岸领域,弹炸人2222,2017
4,"[Action, Adventure, Simulation]",http://store.steampowered.com/app/772540/Battl...,http://steamcommunity.com/app/772540/reviews/?...,3.99,0,772540,Trickjump Games Ltd,Battle Royale Trainer,2018
